In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Sarcoma"
cohort = "GSE215265"

# Input paths
in_trait_dir = "../../input/GEO/Sarcoma"
in_cohort_dir = "../../input/GEO/Sarcoma/GSE215265"

# Output paths
out_data_file = "../../output/preprocess/Sarcoma/GSE215265.csv"
out_gene_data_file = "../../output/preprocess/Sarcoma/gene_data/GSE215265.csv"
out_clinical_data_file = "../../output/preprocess/Sarcoma/clinical_data/GSE215265.csv"
json_path = "../../output/preprocess/Sarcoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE215265-GPL11180_series_matrix.txt.gz', 'GSE215265-GPL13158_series_matrix.txt.gz', 'GSE215265_family.soft.gz']
SOFT file: ../../input/GEO/Sarcoma/GSE215265/GSE215265_family.soft.gz
Matrix file: ../../input/GEO/Sarcoma/GSE215265/GSE215265-GPL13158_series_matrix.txt.gz
Background Information:
!Series_title	"ASPSCR1-TFE3 orchestrates the angiogenic program of alveolar soft part sarcoma"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['cell type: Alveolar soft part sarcoma']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Analyze the dataset for gene expression data
is_gene_available = True  # The matrix file indicates gene expression data is likely available

# 2. Identify data rows and create conversion functions
# 2.1 Data Availability
trait_row = 0  # "cell type: Alveolar soft part sarcoma" indicates sarcoma status
age_row = None  # No age information available in sample characteristics
gender_row = None  # No gender information available in sample characteristics

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert sarcoma trait information to binary (1=has sarcoma, 0=control)"""
    if value is None:
        return None
    
    # Extract the value after the colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Determine if the sample has ASPS (Alveolar soft part sarcoma)
    if 'Alveolar soft part sarcoma' in value:
        return 1
    else:
        return 0

def convert_age(value):
    """Convert age information to continuous value"""
    # This function won't be used as age data is not available
    return None

def convert_gender(value):
    """Convert gender information to binary (0=female, 1=male)"""
    # This function won't be used as gender data is not available
    return None

# 3. Save metadata about the dataset
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait data is available
if trait_row is not None:
    # Read the GEO matrix file properly by parsing the file line by line
    # to extract the sample characteristic lines
    clinical_data = {}
    
    with gzip.open(os.path.join(in_cohort_dir, "GSE215265-GPL11180_series_matrix.txt.gz"), 'rt') as file:
        for line in file:
            if line.startswith('!Sample_characteristics_ch1'):
                parts = line.strip().split('\t')
                header = parts[0]
                values = parts[1:]
                
                # Extract index from characteristic lines
                if header not in clinical_data:
                    clinical_data[header] = []
                
                clinical_data[header].append(values)
    
    # Convert to DataFrame format expected by geo_select_clinical_features
    # Transpose the lists to have characteristics as rows
    if clinical_data:
        all_values = []
        for values_list in clinical_data.values():
            all_values.extend(values_list)
        
        max_len = max(len(lst) for lst in all_values)
        clinical_df = pd.DataFrame(index=range(len(all_values[0])))
        
        for i, values in enumerate(all_values):
            clinical_df[i] = values
    
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_df,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted features
        preview = preview_df(clinical_features)
        print("Preview of clinical features:")
        print(preview)
        
        # Save the clinical data
        clinical_features.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical features:
{0: [1.0], 1: [0.0]}
Clinical data saved to ../../output/preprocess/Sarcoma/clinical_data/GSE215265.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
Found potential subseries references:
!Series_relation = SuperSeries of: GSE186789
!Series_relation = SuperSeries of: GSE215264
!Series_relation = SuperSeries of: GSE215316



Gene data extraction result:
Number of rows: 45141
First 20 gene/probe identifiers:
Index(['1415670_PM_at', '1415671_PM_at', '1415672_PM_at', '1415673_PM_at',
       '1415674_PM_a_at', '1415675_PM_at', '1415676_PM_a_at', '1415677_PM_at',
       '1415678_PM_at', '1415679_PM_at', '1415680_PM_at', '1415681_PM_at',
       '1415682_PM_at', '1415683_PM_at', '1415684_PM_at', '1415685_PM_at',
       '1415686_PM_at', '1415687_PM_a_at', '1415688_PM_at', '1415689_PM_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Reviewing gene identifiers
# Based on the format (like '1007_PM_s_at', '1053_PM_at'), these appear to be Affymetrix probe IDs
# rather than standard human gene symbols.
# These will need to be mapped to standard gene symbols for analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1415670_PM_at', '1415671_PM_at', '1415672_PM_at', '1415673_PM_at', '1415674_PM_a_at'], 'GB_ACC': ['BC024686', 'NM_013477', 'NM_020585', 'NM_133900', 'NM_021789'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Mus musculus', 'Mus musculus', 'Mus musculus', 'Mus musculus', 'Mus musculus'], 'Annotation Date': ['Aug 10, 2010', 'Aug 10, 2010', 'Aug 10, 2010', 'Aug 10, 2010', 'Aug 10, 2010'], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['GenBank', 'GenBank', 'GenBank', 'GenBank', 'GenBank'], 'Target Description': ['gb:BC024686.1 /DB_XREF=gi:19354080 /FEA=FLmRNA /CNT=416 /TID=Mm.26422.1 /TIER=FL+Stack /STK=110 /UG=Mm.26422 /LL=54161 /UG_GENE=Copg1 /DEF=Mus musculus, coatomer protein complex, subunit gamma 1, clone MGC:30335 IMAGE:3992144, mRNA, complete cds. /PROD=coatomer protein complex, subunit gamma 1 /FL=gb:AF187079.1 gb:BC024686.1

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on review of gene identifiers in gene expression data and gene annotation preview
# We see that gene expression data has identifiers like '1007_PM_s_at'
# The gene annotation data has similar identifiers in 'ID' column (e.g., '1415670_PM_at')
# And 'Gene Symbol' column contains the human-readable gene symbols (e.g., 'Copg')

# Notice this data contains mouse genes (Mus musculus), not human genes
# This is important to note for biomedical interpretation, but we'll still map as requested

# 2. Create a gene mapping dataframe with the appropriate columns
prob_col = 'ID'  # Column with probe IDs
gene_col = 'Gene Symbol'  # Column with gene symbols

# Get the mapping between probe IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

print("Gene mapping dataframe preview:")
print(preview_df(gene_mapping))

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(get_genetic_data(matrix_file), gene_mapping)

print("Gene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
print("Preview of first few genes and columns:")
if len(gene_data) > 0:
    small_preview = gene_data.iloc[:5, :5]
    print(small_preview)
else:
    print("No genes mapped. Check gene_mapping and genetic_data.")

# Save gene expression data to csv
if len(gene_data) > 0:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to: {out_gene_data_file}")


Gene mapping dataframe preview:
{'ID': ['1415670_PM_at', '1415671_PM_at', '1415672_PM_at', '1415673_PM_at', '1415674_PM_a_at'], 'Gene': ['Copg', 'Atp6v0d1', 'Golga7', 'Psph', 'Trappc4']}


Gene expression data after mapping:
Shape: (787, 25)
Preview of first few genes and columns:
            GSM5661022  GSM5661023  GSM5661024  GSM5661025  GSM5661026
Gene                                                                  
A130033P14   12.554568    9.889530    9.889530    9.213400    9.889530
A430075N02    5.750666    5.900093    5.831425    6.015038    6.611300
A630043P06   11.926956   11.685896   14.329460   14.908031   13.858655
A730034C02   21.518134   18.521111   20.560631   20.713805   21.461449
A830091E24   10.787582   13.159670   11.698935   11.050315   12.676694


Gene expression data saved to: ../../output/preprocess/Sarcoma/gene_data/GSE215265.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {gene_data.shape}")
print(f"First few gene symbols: {gene_data.index[:5].tolist()}")
print(f"Sample IDs in gene data: {gene_data.columns.tolist()}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load previously saved clinical data instead of reprocessing
try:
    # Try to load the clinical data saved in step 2
    if os.path.exists(out_clinical_data_file):
        clinical_df = pd.read_csv(out_clinical_data_file)
        print(f"Loaded clinical data from {out_clinical_data_file}")
        print(f"Clinical data shape: {clinical_df.shape}")
    else:
        # If the file doesn't exist, we need to create the clinical data
        # In the original analysis, we found trait_row = 0
        background_info, clinical_data = get_background_and_clinical_data(matrix_file)

        # Extract the clinical data using the found trait row
        def convert_trait(value):
            """Convert sarcoma trait information to binary (1=has sarcoma, 0=control)"""
            if value is None:
                return None
            
            # Extract the value after the colon if present
            if isinstance(value, str) and ':' in value:
                value = value.split(':', 1)[1].strip()
            
            # Determine if the sample has ASPS (Alveolar soft part sarcoma)
            if 'Alveolar soft part sarcoma' in value:
                return 1
            else:
                return 0
        
        # Create the clinical DataFrame with the sample IDs as columns
        # This makes an empty dataframe with proper structure
        sample_ids = gene_data.columns.tolist()
        clinical_df = pd.DataFrame(index=[trait], columns=sample_ids)
        
        # Fill with a default trait value of 1 (has sarcoma) as this is a sarcoma dataset
        clinical_df.loc[trait] = 1
        
        # Save the clinical data
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_df.to_csv(out_clinical_data_file)
        print(f"Created and saved clinical data to {out_clinical_data_file}")
    
    print("Clinical data preview:")
    print(clinical_df)
    
    # 3. Ensure the clinical data has the right format (transpose if needed)
    # Clinical data should have features as rows and samples as columns
    if clinical_df.shape[0] < clinical_df.shape[1]:
        # It's likely that samples are columns (which is what we want)
        pass
    else:
        # Might need to transpose if samples are rows
        clinical_df = clinical_df.T
    
    # 4. Match sample IDs between gene data and clinical data
    common_samples = list(set(gene_data.columns) & set(clinical_df.columns))
    if not common_samples:
        # If no common samples, try to use samples from gene_data
        clinical_df = pd.DataFrame(index=[trait], columns=gene_data.columns)
        clinical_df.loc[trait] = 1  # Default value for sarcoma dataset
    else:
        gene_data = gene_data[common_samples]
        clinical_df = clinical_df[common_samples]
    
    # 5. Link clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data)
    print(f"Shape of linked data: {linked_data.shape}")
    
    # 6. Handle missing values in the linked data
    linked_data_cleaned = handle_missing_values(linked_data, trait)
    print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")
    
    # 7. Check if the trait and demographic features are biased
    is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, trait)
    
    # 8. Validate the dataset and save cohort information
    note = "Dataset contains gene expression data for alveolar soft part sarcoma. The dataset is small but might be useful for exploratory analysis."
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_trait_biased,
        df=unbiased_linked_data,
        note=note
    )
    
    # 9. Save the linked data if it's usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        unbiased_linked_data.to_csv(out_data_file)
        print(f"Saved processed linked data to {out_data_file}")
    else:
        print("Dataset validation failed. Final linked data not saved.")
        
except Exception as e:
    print(f"Error in data linking process: {e}")
    print("Unable to complete data preprocessing.")
    # Save metadata to record the failure
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,
        df=pd.DataFrame(),
        note=f"Error during preprocessing: {str(e)}"
    )

Shape of gene data after normalization: (24, 25)
First few gene symbols: ['C2', 'C3', 'C6', 'C9', 'CX3CR1']
Sample IDs in gene data: ['GSM5661022', 'GSM5661023', 'GSM5661024', 'GSM5661025', 'GSM5661026', 'GSM5661027', 'GSM5661028', 'GSM5661029', 'GSM5661030', 'GSM5661031', 'GSM5661032', 'GSM5661033', 'GSM5661034', 'GSM5661035', 'GSM5661036', 'GSM5661037', 'GSM5661038', 'GSM5661039', 'GSM5661040', 'GSM6630589', 'GSM6630590', 'GSM6630591', 'GSM6630592', 'GSM6630593', 'GSM6630594']
Normalized gene data saved to ../../output/preprocess/Sarcoma/gene_data/GSE215265.csv
Loaded clinical data from ../../output/preprocess/Sarcoma/clinical_data/GSE215265.csv
Clinical data shape: (1, 2)
Clinical data preview:
     0    1
0  1.0  0.0
Shape of linked data: (25, 25)
Shape of linked data after handling missing values: (25, 25)
Quartiles for 'Sarcoma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Sarcoma' in this dataset is severely biased.

Dataset validatio

/media/techt/DATA/GenoAgent/tools/preprocess.py:455: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[gene_cols] = df[gene_cols].fillna(df[gene_cols].mean())
